In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 16.8 MB/s eta 0:00:00


In [15]:
"""Reward function in conversation will be made up of three components:
- r_c = congruence reward: how likely is the agent to have said what the respondent said (negative KL divergence of the next token probabilities)
- r_s = sentiment reward: how positive was the sentiment of the respondent (use a pre-existing sentiment model)
- r_a = affection reward: how much does the agent like the respondent (use discounted sum of previous rewards)

r=r_c*r_s*r_a + epsilon, where epsilon is some very small noise term
"""

import os
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from google.colab import drive
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification


def get_congruence_reward(comment_ids, response_ids_trunc, agent_model):
    """Iterate through respondent's response to agent's comment, adding each token to the
    prompt each time, and get the KL divergence for what the agent would have said instead.
    Take the mean of all KL divergences at the end to give congruence reward.

    Args:
    comment_ids (torch tensor): IDs of agent's comment, including original query.
    response_ids_trunc (torch tensor): IDs of respondent's response, not including
        original comment or query
    agent_model (transformers GPT2LMHeadModel): the agent model

    Returns:
    float: Congruence reward value
    """

    KL_divs = list()
    target_ids = comment_ids.clone()

    for id in response_ids_trunc[0]:
        agent_output = agent(target_ids)
        agent_log_probs = F.log_softmax(agent_output.logits[0][-1], dim=0) # Predicted log probs
        KL_div = -agent_log_probs[id] # Calculate KL divergence (same as cross-entropy in this case)
        KL_divs.append(KL_div.item())
        target_ids = torch.cat((target_ids.squeeze(0), id.unsqueeze(0)), dim=0).unsqueeze(0)

    mean_div = np.mean(KL_divs)
    congruence_reward = np.sqrt(np.exp(-mean_div))

    return congruence_reward


def get_sentiment_reward(response_text_trunc, sentiment_tokenizer, sentiment_model):
    """Get a scalar reward corresponding to sentiment of respondent's response.

    Args:
    response_text_trunc (torch tensor): text of respondent's response, not including
        original comment or query
    sentiment_tokenizer (transformers AutoTokenizer): tokenizer for sentiment model
    sentiment_tokenizer (transformers AutoModelForSequenceClassification):
        sentiment model

    Returns:
    float: Sentiment reward value
    """

    # Get sentiment probabilities from model (negative, neutral, or positive)
    sentiment_response_ids = sentiment_tokenizer.encode(response_text_trunc, return_tensors="pt").to('cuda')
    sentiment_probs = F.softmax(sentiment_model(sentiment_response_ids).logits.detach(), dim=1)[0]

    # Calculate the reward as the positive probability minus the negative probability
    sentiment_reward = (sentiment_probs[2] - sentiment_probs[0]).item()

    # Scale to be between 0 and 1
    sentiment_reward = (sentiment_reward + 1) / 2

    return sentiment_reward


def get_affection_reward(affection_counter):
    """Given by cube root of affection_counter."""
    affection_reward = np.cbrt(affection_counter)
    return affection_reward


def update_affection_counter(affection_counter, last_raw_reward, discount=0.1):
    """Update scalar affection counter with discounted most recent raw
    reward, i.e. reward before scaling away from between 0 and 1.

    Args:
        affection_counter (float): Long-term value for liking other agent
        last_raw_reward (float): Number between 0 and 1 from last unprocessed
            reward
        discount (float): Fraction deciding how fast to update counter

    Returns:
        float: Updated long-term value for liking other agent
    """

    affection_counter = (1-discount)*affection_counter + discount*last_raw_reward
    return affection_counter


def raw_reward_combiner(congruence_reward, sentiment_reward, affection_reward, epsilon=0.01):
    """Combine all three reward terms and add noise.

    Args:
        congruence_reward (float): Congruence reward value
        sentiment_reward (float): Sentiment reward value
        affection_reward (float): Affection reward value
        epsilon (float): Scaling factor to apply to exponentially distributed noise term

    Returns:
        float: Combined raw reward between 0 and 1
    """

    noise_term = epsilon*np.random.exponential(1)
    combined_raw_reward = congruence_reward*sentiment_reward*affection_reward + noise_term
    return combined_raw_reward


def scale_combined_reward(combined_raw_reward, amplitude=10):
    """Scale reward values by some method to make them work better.
    For now will just multiply by a constant factor.

    Args:
        combined_raw_reward (float): Combined raw reward between 0 and 1
        amplitude (float): Size of scaling factor to multiply by

    Returns:
        float: Combined scaled reward
    """

    combined_scaled_reward = amplitude*combined_raw_reward
    return combined_scaled_reward


drive.mount('/content/drive')
project_path = './drive/MyDrive/Colab Notebooks/GPT_community/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Load in prompts
prompts_file = os.path.join(project_path, 'data/brighton_philosophy_prompts.txt')
with open(prompts_file) as file:
    prompts = [line.rstrip() for line in file]

# Create agent and respondent models
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
pad_token_id = tokenizer.eos_token_id
agent = GPT2LMHeadModel.from_pretrained('Linus4Lyf/Kant_Metaphysics_Of_Morals').to('cuda')
respondent = GPT2LMHeadModel.from_pretrained('Linus4Lyf/Hume_A_Treatise_Of_Human_Nature').to('cuda')

# Create sentiment model
sent_tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
sent_model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment").to('cuda')

# Initialise affection counter at neutral value
affection_counter = np.power(0.5, 3)

In [21]:
questioner_name = 'Socrates'
agent_name = 'Kant'
respondent_name = 'Hume'

# Get query from questions list
query_text = f"{questioner_name}: " + np.random.choice(prompts)
print(query_text, '\n')
query_text += f"\n{agent_name}: "

# Encode query and get comment from agent
query_ids = tokenizer.encode(query_text, return_tensors='pt').to('cuda')
comment_ids = agent.generate(query_ids, do_sample=True, temperature=0.9, max_new_tokens=200, pad_token_id=pad_token_id, eos_token_id=pad_token_id)
comment_text = tokenizer.batch_decode(comment_ids)[0]
print('--------------------------------------------------------------------------------------')
print(comment_text, '\n')
comment_text += f"\n{respondent_name}: "

# Get response from respondent
comment_ids = tokenizer.encode(comment_text, return_tensors='pt').to('cuda')
response_ids = respondent.generate(comment_ids, do_sample=True, temperature=0.9, max_new_tokens=200, pad_token_id=pad_token_id, eos_token_id=pad_token_id)
response_text = tokenizer.batch_decode(response_ids)[0]
print('--------------------------------------------------------------------------------------')
print(response_text, '\n')

# Remove original query and comment from response text
response_text_trunc = response_text.replace(comment_text, '')[1:]
response_ids_trunc = tokenizer.encode(response_text_trunc, return_tensors='pt').to('cuda')
print('--------------------------------------------------------------------------------------')
print(response_text_trunc, '\n')

# Get reward for response
congruence_reward = get_congruence_reward(comment_ids, response_ids_trunc, agent)
sentiment_reward = get_sentiment_reward(response_text_trunc, sent_tokenizer, sent_model)
affection_reward = get_affection_reward(affection_counter)
print(f"Congruence reward = {congruence_reward}, Sentiment reward = {sentiment_reward}, Affection reward = {affection_reward}")

# Combine components of reward
combined_raw_reward = raw_reward_combiner(congruence_reward, sentiment_reward, affection_reward, epsilon=0.01)
print(f"Combined raw reward = {combined_raw_reward}")

# Update affection counter
affection_counter = update_affection_counter(affection_counter, combined_raw_reward, discount=0.1)
print(f"New affection counter = {affection_counter}")

# Calculate scaled reward value
final_reward = scale_combined_reward(combined_raw_reward, amplitude=10)
print(f"Final reward value = {final_reward}")

Socrates: Is religion a force for good? 

--------------------------------------------------------------------------------------
Socrates: Is religion a force for good?
Kant:  Is it possible that the most efficient of all forms of religion is that which cannot be used for good by any other or general good besides itself; and since that which is absolutely the only good which can ever be done and which is the only good which can be made, must always be the best which can ever be obtained by others, that is to say, by a certain amount of virtue, it follows that its efficacy depends on all the good which has been done; that is to say it is just to make to itself a certain proportion of it, which of itself consists, the best in its essence, the best in its causes, and that it is the only good which can ever be practised, because it is called virtue because it is so by virtue, and so alone is its duty to make itself an object of its own good, since that which consists of itself, by virtue, 